This notebook cleans 311 complaints related to heat and hot water from 2020 to 2022.

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%R

require('tidyverse')
require('ggrepel')
require('ggpubr')
require('broom')
require('AICcmodavg')

R[write to console]: Loading required package: tidyverse



── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   1.0.1 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.3.0      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 1.0.0 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


R[write to console]: Loading required package: ggrepel

R[write to console]: Loading required package: ggpubr

R[write to console]: Loading required package: broom

R[write to console]: Loading required package: AICcmodavg



In [3]:
# read csv
# df = pd.read_csv('311-heat-complaints.csv')
# df

In [4]:
# df.to_parquet('311-heat-complaints.parquet')

In [5]:
df = pd.read_parquet('311-heat-complaints.parquet')
df

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,45278545,12/31/2019 12:01:28 AM,01/02/2020 06:07:39 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10009.0,73 AVENUE C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.722955,-73.979692,"(40.72295450944238, -73.97969228924251)"
1,45284487,12/31/2019 12:04:42 AM,01/01/2020 06:33:28 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10463.0,2700 KINGSBRIDGE TERRACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.870736,-73.903938,"(40.87073636320048, -73.90393774477785)"
2,45282424,12/31/2019 12:04:46 AM,01/03/2020 09:12:00 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11692.0,56-16 BEACH CHANNEL DRIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.594904,-73.787079,"(40.594904470195445, -73.78707871841593)"
3,45283831,12/31/2019 12:05:17 AM,12/31/2019 08:41:49 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11217.0,30 3 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.685810,-73.980385,"(40.68580972753562, -73.98038507594089)"
4,45279226,12/31/2019 12:07:28 AM,01/02/2020 08:07:31 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11207.0,502 WILLIAMS AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.663489,-73.897586,"(40.66348921631345, -73.89758594290396)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632339,56409507,12/31/2022 11:43:27 PM,01/03/2023 08:51:31 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10037.0,2171 MADISON AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.812628,-73.935613,"(40.81262819922922, -73.93561342471233)"
632340,56408321,12/31/2022 11:49:10 PM,01/04/2023 08:21:34 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11203.0,530 EAST 56 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.643689,-73.923709,"(40.643689430579094, -73.92370862728106)"
632341,56409234,12/31/2022 11:52:15 PM,01/04/2023 04:26:07 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11385.0,1713 HARMAN STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.705020,-73.913767,"(40.705019921035245, -73.91376683177621)"
632342,56408901,12/31/2022 11:55:17 PM,01/04/2023 03:09:14 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10459.0,830 BECK STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.817780,-73.897218,"(40.8177796548477, -73.89721844090171)"


In [6]:
df = df.drop(['Cross Street 1', 'Cross Street 2', 'Intersection Street 1', 'Intersection Street 2', 'Landmark', 'Facility Type', 'Due Date', 'BBL', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough', 'Park Borough', 'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment', 'Open Data Channel Type'], axis=1)
df = df.rename(columns={'Unique Key': 'unique_key', 'Created Date': 'created_date', 'Closed Date': 'closed_date', 'Agency': 'agency', 'Agency Name': 'agency_name', 'Complaint Type': 'complaint_type', 'Descriptor': 'descriptor', 'Location Type': 'location_type', 'Incident Zip': 'zip', 'Incident Address': 'incident_address', 'Street Name': 'street', 'Address Type': 'address_type', 'City': 'city', 'Status': 'status', 'Resolution Description': 'resolution_desc', 'Resolution Action Updated Date': 'resolution_action_updated_date', 'Community Board': 'community_board', 'Borough': 'borough', 'Latitude': 'lat', 'Longitude': 'long', 'Location': 'location'})
df

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,zip,incident_address,...,address_type,city,status,resolution_desc,resolution_action_updated_date,community_board,borough,lat,long,location
0,45278545,12/31/2019 12:01:28 AM,01/02/2020 06:07:39 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10009.0,73 AVENUE C,...,ADDRESS,NEW YORK,Closed,The Department of Housing Preservation and Dev...,01/02/2020 06:07:39 AM,03 MANHATTAN,MANHATTAN,40.722955,-73.979692,"(40.72295450944238, -73.97969228924251)"
1,45284487,12/31/2019 12:04:42 AM,01/01/2020 06:33:28 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10463.0,2700 KINGSBRIDGE TERRACE,...,ADDRESS,BRONX,Closed,The Department of Housing Preservation and Dev...,01/01/2020 06:33:28 PM,08 BRONX,BRONX,40.870736,-73.903938,"(40.87073636320048, -73.90393774477785)"
2,45282424,12/31/2019 12:04:46 AM,01/03/2020 09:12:00 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11692.0,56-16 BEACH CHANNEL DRIVE,...,ADDRESS,Arverne,Closed,The Department of Housing Preservation and Dev...,01/03/2020 09:12:00 AM,14 QUEENS,QUEENS,40.594904,-73.787079,"(40.594904470195445, -73.78707871841593)"
3,45283831,12/31/2019 12:05:17 AM,12/31/2019 08:41:49 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11217.0,30 3 AVENUE,...,ADDRESS,BROOKLYN,Closed,The Department of Housing Preservation and Dev...,12/31/2019 08:41:49 PM,02 BROOKLYN,BROOKLYN,40.685810,-73.980385,"(40.68580972753562, -73.98038507594089)"
4,45279226,12/31/2019 12:07:28 AM,01/02/2020 08:07:31 AM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11207.0,502 WILLIAMS AVENUE,...,ADDRESS,BROOKLYN,Closed,The complaint you filed is a duplicate of a co...,01/02/2020 08:07:31 AM,05 BROOKLYN,BROOKLYN,40.663489,-73.897586,"(40.66348921631345, -73.89758594290396)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632339,56409507,12/31/2022 11:43:27 PM,01/03/2023 08:51:31 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10037.0,2171 MADISON AVENUE,...,ADDRESS,NEW YORK,Closed,The Department of Housing Preservation and Dev...,01/03/2023 12:00:00 AM,11 MANHATTAN,MANHATTAN,40.812628,-73.935613,"(40.81262819922922, -73.93561342471233)"
632340,56408321,12/31/2022 11:49:10 PM,01/04/2023 08:21:34 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11203.0,530 EAST 56 STREET,...,ADDRESS,BROOKLYN,Closed,The Department of Housing Preservation and Dev...,01/04/2023 12:00:00 AM,17 BROOKLYN,BROOKLYN,40.643689,-73.923709,"(40.643689430579094, -73.92370862728106)"
632341,56409234,12/31/2022 11:52:15 PM,01/04/2023 04:26:07 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11385.0,1713 HARMAN STREET,...,ADDRESS,RIDGEWOOD,Closed,The Department of Housing Preservation and Dev...,01/04/2023 12:00:00 AM,05 QUEENS,QUEENS,40.705020,-73.913767,"(40.705019921035245, -73.91376683177621)"
632342,56408901,12/31/2022 11:55:17 PM,01/04/2023 03:09:14 PM,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10459.0,830 BECK STREET,...,ADDRESS,BRONX,Closed,The Department of Housing Preservation and Dev...,01/04/2023 12:00:00 AM,02 BRONX,BRONX,40.817780,-73.897218,"(40.8177796548477, -73.89721844090171)"


In [7]:
df['created_time'] = df['created_date'].str.extract(r'(\d{2}:\d{2}:\d{2} [AP]M)')
df['created_date'] = df['created_date'].str.extract(r'(\d{2}/\d{2}/\d{4})')
df['closed_time'] = df['closed_date'].str.extract(r'(\d{2}:\d{2}:\d{2} [AP]M)')
df['closed_date'] = df['closed_date'].str.extract(r'(\d{2}/\d{2}/\d{4})')
df['created_date'] = pd.to_datetime(df['created_date'], format='%m/%d/%Y')
df['closed_date'] = pd.to_datetime(df['closed_date'], format='%m/%d/%Y')
df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,zip,incident_address,...,status,resolution_desc,resolution_action_updated_date,community_board,borough,lat,long,location,created_time,closed_time
0,45278545,2019-12-31,2020-01-02,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10009.0,73 AVENUE C,...,Closed,The Department of Housing Preservation and Dev...,01/02/2020 06:07:39 AM,03 MANHATTAN,MANHATTAN,40.722955,-73.979692,"(40.72295450944238, -73.97969228924251)",12:01:28 AM,06:07:39 AM
1,45284487,2019-12-31,2020-01-01,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10463.0,2700 KINGSBRIDGE TERRACE,...,Closed,The Department of Housing Preservation and Dev...,01/01/2020 06:33:28 PM,08 BRONX,BRONX,40.870736,-73.903938,"(40.87073636320048, -73.90393774477785)",12:04:42 AM,06:33:28 PM
2,45282424,2019-12-31,2020-01-03,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11692.0,56-16 BEACH CHANNEL DRIVE,...,Closed,The Department of Housing Preservation and Dev...,01/03/2020 09:12:00 AM,14 QUEENS,QUEENS,40.594904,-73.787079,"(40.594904470195445, -73.78707871841593)",12:04:46 AM,09:12:00 AM
3,45283831,2019-12-31,2019-12-31,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11217.0,30 3 AVENUE,...,Closed,The Department of Housing Preservation and Dev...,12/31/2019 08:41:49 PM,02 BROOKLYN,BROOKLYN,40.685810,-73.980385,"(40.68580972753562, -73.98038507594089)",12:05:17 AM,08:41:49 PM
4,45279226,2019-12-31,2020-01-02,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11207.0,502 WILLIAMS AVENUE,...,Closed,The complaint you filed is a duplicate of a co...,01/02/2020 08:07:31 AM,05 BROOKLYN,BROOKLYN,40.663489,-73.897586,"(40.66348921631345, -73.89758594290396)",12:07:28 AM,08:07:31 AM


In [8]:
df['year'] = df['created_date'].dt.year
df['month'] = df['created_date'].dt.month
df['month_year'] = df['created_date'].dt.to_period('M')
df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,zip,incident_address,...,community_board,borough,lat,long,location,created_time,closed_time,year,month,month_year
0,45278545,2019-12-31,2020-01-02,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10009.0,73 AVENUE C,...,03 MANHATTAN,MANHATTAN,40.722955,-73.979692,"(40.72295450944238, -73.97969228924251)",12:01:28 AM,06:07:39 AM,2019,12,2019-12
1,45284487,2019-12-31,2020-01-01,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,10463.0,2700 KINGSBRIDGE TERRACE,...,08 BRONX,BRONX,40.870736,-73.903938,"(40.87073636320048, -73.90393774477785)",12:04:42 AM,06:33:28 PM,2019,12,2019-12
2,45282424,2019-12-31,2020-01-03,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11692.0,56-16 BEACH CHANNEL DRIVE,...,14 QUEENS,QUEENS,40.594904,-73.787079,"(40.594904470195445, -73.78707871841593)",12:04:46 AM,09:12:00 AM,2019,12,2019-12
3,45283831,2019-12-31,2019-12-31,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11217.0,30 3 AVENUE,...,02 BROOKLYN,BROOKLYN,40.685810,-73.980385,"(40.68580972753562, -73.98038507594089)",12:05:17 AM,08:41:49 PM,2019,12,2019-12
4,45279226,2019-12-31,2020-01-02,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11207.0,502 WILLIAMS AVENUE,...,05 BROOKLYN,BROOKLYN,40.663489,-73.897586,"(40.66348921631345, -73.89758594290396)",12:07:28 AM,08:07:31 AM,2019,12,2019-12


In [9]:
# remove 2019 
df = df[df['year'] != 2019]

In [10]:
# remove decimal from zip
df['zip'] = df['zip'].astype(str).str[:-2]
df.head(1)

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,zip,incident_address,...,community_board,borough,lat,long,location,created_time,closed_time,year,month,month_year
608,45288075,2020-01-01,2020-01-03,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11238,420 CLINTON AVENUE,...,02 BROOKLYN,BROOKLYN,40.685645,-73.967333,"(40.68564482086869, -73.96733257810193)",12:04:45 AM,02:12:58 AM,2020,1,2020-01


In [11]:
# how many rows don't have lat/long
df[df['lat'].isnull()].shape

(107, 26)

In [13]:
# group by month and look number of complaints in address Tiebout Avenue in 2022
df[df['incident_address'] == '2176 TIEBOUT AVENUE'].groupby('month_year').count()['unique_key']

month_year
2020-01    602
2020-02    671
2020-03    457
2020-04    334
2020-05    384
2020-06     92
2020-07      2
2020-10     45
2020-11    246
2020-12    409
2021-01    528
2021-02    832
2021-03    676
2021-04    450
2021-05    202
2021-06     10
2021-07      6
2021-10      6
2021-11    306
2021-12    691
2022-01    764
2022-02    768
2022-03    665
2022-04    600
2022-05    239
2022-06    130
2022-07      1
2022-08      3
2022-09      4
2022-10      7
2022-11     38
2022-12     91
Freq: M, Name: unique_key, dtype: int64

In [ ]:
# look at 2176 Tiebout Avenue in 2022
df_tiebout = df[(df['incident_address'] == '2176 TIEBOUT AVENUE') & (df['year'] == 2022)]
df_tiebout

In [ ]:
# save to csv

# df_tiebout.to_csv('2176_tiebout.csv', index=False)

Dataframe 1: Address level data

In [14]:
df_address = df[(df['agency'] == 'HPD')].groupby(['year', 'borough', 'zip', 'incident_address', 'lat', 'long']).size().reset_index(name='count')
df_address


,year,borough,zip,incident_address,lat,long,count
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...
106127,2022,STATEN ISLAND,10314,81 CHURCH AVENUE,40.592844,-74.185280,1
106128,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587616,-74.165297,1
106129,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587651,-74.165359,5
106130,2022,STATEN ISLAND,10314,98 NOSTRAND AVENUE,40.606914,-74.169706,1


In [15]:
# are there any zip codes that are null?
df_address[df_address['zip'].isnull()]


,year,borough,zip,incident_address,lat,long,count


In [16]:
df_address = df_address.rename(columns={'count': 'num_complaints'})
df_address


,year,borough,zip,incident_address,lat,long,num_complaints
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...
106127,2022,STATEN ISLAND,10314,81 CHURCH AVENUE,40.592844,-74.185280,1
106128,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587616,-74.165297,1
106129,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587651,-74.165359,5
106130,2022,STATEN ISLAND,10314,98 NOSTRAND AVENUE,40.606914,-74.169706,1


In [17]:
# save to csv
# df_address.to_csv('address_level.csv', index=False)

In [37]:
# save to parquet
df_address.to_parquet('address_level.parquet')

In [18]:
# which address has the most complaints?
df_address.groupby(['year', 'borough', 'zip', 'incident_address']).agg({'num_complaints': 'sum'}).reset_index().sort_values(by=['num_complaints'], ascending=False)



,year,borough,zip,incident_address,num_complaints
30358,2021,BRONX,10457,2176 TIEBOUT AVENUE,3707
63445,2022,BRONX,10457,2176 TIEBOUT AVENUE,3310
2220,2020,BRONX,10457,2176 TIEBOUT AVENUE,3242
61284,2022,BRONX,10452,957 WOODYCREST AVENUE,1423
24356,2020,QUEENS,11373,89-21 ELMHURST AVENUE,1369
...,...,...,...,...,...
46250,2021,BROOKLYN,11238,372 FRANKLIN AVENUE,1
46252,2021,BROOKLYN,11238,375 FLATBUSH AVENUE,1
46253,2021,BROOKLYN,11238,375 GRAND AVENUE,1
46258,2021,BROOKLYN,11238,388 ST MARKS AVENUE,1


In [19]:
# check for complaints from zip code 10457
df_address[df_address['zip'] == '10457'].sort_values(by=['num_complaints'], ascending=False)

,year,borough,zip,incident_address,lat,long,num_complaints
30935,2021,BRONX,10457,2176 TIEBOUT AVENUE,40.854439,-73.898423,3707
67181,2022,BRONX,10457,2176 TIEBOUT AVENUE,40.854439,-73.898423,3310
2235,2020,BRONX,10457,2176 TIEBOUT AVENUE,40.854439,-73.898423,3242
67023,2022,BRONX,10457,2000 ANTHONY AVENUE,40.851443,-73.902863,296
67099,2022,BRONX,10457,2082 HUGHES AVENUE,40.848514,-73.890697,247
...,...,...,...,...,...,...,...
30614,2021,BRONX,10457,1685 TOPPING AVENUE,40.843799,-73.905852,1
66918,2022,BRONX,10457,1800 MONROE AVENUE,40.846862,-73.906470,1
30608,2021,BRONX,10457,1668 GRAND CONCOURSE,40.843680,-73.911758,1
66921,2022,BRONX,10457,1805 CLINTON AVENUE,40.842195,-73.892612,1


Dataframe 2: Zip level data merged with census data

In [20]:
df_address

,year,borough,zip,incident_address,lat,long,num_complaints
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...
106127,2022,STATEN ISLAND,10314,81 CHURCH AVENUE,40.592844,-74.185280,1
106128,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587616,-74.165297,1
106129,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587651,-74.165359,5
106130,2022,STATEN ISLAND,10314,98 NOSTRAND AVENUE,40.606914,-74.169706,1


In [21]:
# group by year and borough, see num_complaints

df_address.groupby(['year', 'borough']).agg({'num_complaints': 'sum'})

num_complaints
year borough                      
2020 BRONX                   58953
     BROOKLYN                46011
     MANHATTAN               36320
     QUEENS                  22053
     STATEN ISLAND            1653
2021 BRONX                   69800
     BROOKLYN                56317
     MANHATTAN               44402
     QUEENS                  26750
     STATEN ISLAND            2278
2022 BRONX                   96178
     BROOKLYN                71183
     MANHATTAN               58713
     QUEENS                  35256
     STATEN ISLAND            3050

In [22]:
# manually checking for errors in the 311 data and missing data in census 
df_address[(df_address['zip'] == '11001') & (df_address['borough'] == 'QUEENS')]


,year,borough,zip,incident_address,lat,long,num_complaints
22345,2020,QUEENS,11001,86-16 263 STREET,40.733931,-73.704105,1
56351,2021,QUEENS,11001,84-34 260 STREET,40.736313,-73.707952,1
56352,2021,QUEENS,11001,85-08 LITTLE NECK PARKWAY,40.733952,-73.712357,1
56353,2021,QUEENS,11001,87-41 LITTLE NECK PARKWAY,40.729779,-73.710676,2
56354,2021,QUEENS,11001,87-45 LITTLE NECK PARKWAY,40.729693,-73.710669,1
97221,2022,QUEENS,11001,264-16 85 AVENUE,40.736096,-73.703627,1
97222,2022,QUEENS,11001,84-16 LITTLE NECK PARKWAY,40.735852,-73.713526,2
97223,2022,QUEENS,11001,84-74 LITTLE NECK PARKWAY,40.734513,-73.712705,2
97224,2022,QUEENS,11001,86-14 LITTLE NECK PARKWAY,40.732116,-73.711344,1
97225,2022,QUEENS,11001,87-41 LITTLE NECK PARKWAY,40.729397,-73.710645,1


I found several errors in the 311 data while checking the csv manually. For instance, several zip codes were assigned incorrect boroughs.

I used these websites - https://www.unitedstateszipcodes.org/ and https://www.newyork-demographics.com/ - for zip codes with significant number of complaints but population or income census or both missing. 

In two zip codes, data for 2020 was not available in any of the source hence, 2021 data was considered for all three years.


In [23]:
# replace borough as 'Bronx' for zip code '10463' - error in 311 data
df_address.loc[(df_address['zip'] == '10463') & (df_address['borough'] == 'MANHATTAN'), 'borough'] = 'BRONX'

In [24]:
df_address[(df_address['zip'] == '10463') & (df_address['borough'] == 'MANHATTAN')]

,year,borough,zip,incident_address,lat,long,num_complaints


In [25]:
# how many unique zip codes are there?
df_address['zip'].nunique()

191

In [26]:
# group by zip and year and put in a new dataframe
df_zip = df_address.groupby(['year', 'zip', 'borough']).agg({'num_complaints': 'sum'}).reset_index()
df_zip

,year,zip,borough,num_complaints
0,2020,10001,MANHATTAN,289
1,2020,10002,MANHATTAN,1067
2,2020,10003,MANHATTAN,824
3,2020,10004,MANHATTAN,8
4,2020,10005,MANHATTAN,3
...,...,...,...,...
553,2022,11691,QUEENS,1619
554,2022,11692,QUEENS,470
555,2022,11693,QUEENS,146
556,2022,11694,QUEENS,445


In [27]:
# group by year and borough, see num_complaints

df_zip.groupby(['year', 'borough']).agg({'num_complaints': 'sum'})

num_complaints
year borough                      
2020 BRONX                   59145
     BROOKLYN                46011
     MANHATTAN               36128
     QUEENS                  22053
     STATEN ISLAND            1653
2021 BRONX                   70138
     BROOKLYN                56317
     MANHATTAN               44064
     QUEENS                  26750
     STATEN ISLAND            2278
2022 BRONX                   96515
     BROOKLYN                71183
     MANHATTAN               58376
     QUEENS                  35256
     STATEN ISLAND            3050

In [28]:
df_zip.dtypes

year               int64
zip               object
borough           object
num_complaints     int64
dtype: object

In [29]:
# convert zip to int
df_zip['zip'] = df_zip['zip'].astype(int)

In [32]:
# read census data
# df_census = pd.read_csv('census_data.csv')
# df_census

,zip,borough,pop_2020,pop_2021,med_inc_2020,med_inc_2021,pop_2022,med_inc_2022
0,10001,Manhattan,25026.0,26966.0,96787.0,101409.0,26966.0,101409.0
1,10002,Manhattan,74363.0,76807.0,35607.0,37093.0,76807.0,37093.0
2,10003,Manhattan,54671.0,54447.0,129981.0,137533.0,54447.0,137533.0
3,10004,Manhattan,3310.0,4795.0,204949.0,216017.0,4795.0,216017.0
4,10005,Manhattan,8664.0,8637.0,184681.0,197188.0,8637.0,197188.0
...,...,...,...,...,...,...,...,...
475,10309,Staten,33896.0,35832.0,102730.0,107500.0,35832.0,107500.0
476,10310,Staten,24168.0,25976.0,86895.0,96161.0,25976.0,96161.0
477,10311,Staten,0.0,0.0,-666666666.0,-666666666.0,0.0,-666666666.0
478,10312,Staten,61114.0,63935.0,96785.0,100875.0,63935.0,100875.0


In [33]:
# df_census.to_parquet('census_data.parquet')

In [34]:
df_census = pd.read_parquet('census_data.parquet')
df_census

,zip,borough,pop_2020,pop_2021,med_inc_2020,med_inc_2021,pop_2022,med_inc_2022
0,10001,Manhattan,25026.0,26966.0,96787.0,101409.0,26966.0,101409.0
1,10002,Manhattan,74363.0,76807.0,35607.0,37093.0,76807.0,37093.0
2,10003,Manhattan,54671.0,54447.0,129981.0,137533.0,54447.0,137533.0
3,10004,Manhattan,3310.0,4795.0,204949.0,216017.0,4795.0,216017.0
4,10005,Manhattan,8664.0,8637.0,184681.0,197188.0,8637.0,197188.0
...,...,...,...,...,...,...,...,...
475,10309,Staten,33896.0,35832.0,102730.0,107500.0,35832.0,107500.0
476,10310,Staten,24168.0,25976.0,86895.0,96161.0,25976.0,96161.0
477,10311,Staten,0.0,0.0,-666666666.0,-666666666.0,0.0,-666666666.0
478,10312,Staten,61114.0,63935.0,96785.0,100875.0,63935.0,100875.0


In [35]:
df_census.dtypes


zip               int64
borough          object
pop_2020        float64
pop_2021        float64
med_inc_2020    float64
med_inc_2021    float64
pop_2022        float64
med_inc_2022    float64
dtype: object

In [36]:
# save as csv
# df_zip.to_csv('311_zip_level.csv', index=False)



In [41]:
df_zip.to_parquet('311_zip_level.parquet')